In [1]:
import os
import json
import numpy as np
import torch
from torch.utils.data import DataLoader
from monai.transforms import Compose, LoadImaged, NormalizeIntensityd, ConvertToMultiChannelBasedOnBratsClassesd
from monai.data import Dataset
from monai.inferers import sliding_window_inference
from functools import partial
import matplotlib.pyplot as plt
import nibabel as nib
from monai.metrics import DiceMetric, compute_meandice

import shutil
import tempfile
import time
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR
from monai import data
from monai.data import decollate_batch
from functools import partial

import torch
from sklearn.metrics import jaccard_score

# Load JSON file
with open('/root/swinUNETR/jsons/dataset_validation.json', 'r') as f:
    test_files = json.load(f)
test_files = test_files["training"]
test_transform = Compose([
    LoadImaged(keys=["image", "label"]),
    ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True)
])

test_ds = Dataset(data=test_files, transform=test_transform)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=8, pin_memory=True)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming model and device setup are done elsewhere
model = SwinUNETR(
    img_size=(128, 128, 128),
    in_channels=4,
    out_channels=3,
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.0,
    use_checkpoint=True,
).to(device)

model.load_state_dict(torch.load("/root/swinUNETR/runs/unetr_test_dir/model_final.pt")["state_dict"])
model.to(device)
model.eval()

# Initialize the model inference function
model_inferer = partial(sliding_window_inference, roi_size=[160, 160, 160], sw_batch_size=1, predictor=model, overlap=0.5)
iou_sum = 0
dice_sum = 0
# Compute metrics and visualize
for batch_data in test_loader:
    with torch.no_grad():
        # Prepare data
        images, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        # Model prediction
        outputs = model_inferer(images)
        outputs = torch.sigmoid(outputs)
        # Convert to one-hot format for comparison
        outputs = (outputs > 0.5).float()
        
        # Compute metrics
        labels_flat = labels.view(-1).cpu().numpy()
        outputs_flat = outputs.view(-1).cpu().numpy()
        iou = jaccard_score(labels_flat, outputs_flat, average='binary')
        dice = 2 * iou / (iou + 1)

        print(f"Dice Score: {dice}, IoU: {iou}")
        dice_sum += dice
        iou_sum += iou


/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Dice Score: 0.892223983163406, IoU: 0.8054191186691998


/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Dice Score: 0.9292022645393722, IoU: 0.8677663705925328


/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Dice Score: 0.934781617787826, IoU: 0.8775492738367267


/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/validation/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print(dice_sum, iou_sum)

66.11297798901796 66.11297798901796


In [ ]:
directory = "/root/dataset_val/MICCAI_BraTS2020_TrainingData"
subdirectories = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
num_directories = len(subdirectories)

In [ ]:
dice_score = dice_sum / num_directories
iou_score = iou_sum / num_directories
print(dice_score, iou_score)